In [3]:
# pip install git2vec
from git2vec import loader

In [4]:
# using a random repo as an example
repo_name = "https://github.com/voynow/turbo-docs"

# return a list of Document objects
repo_data = loader.load(repo_name)

# or return a string of all the raw text
raw_repo = loader.load(repo_name, return_str=True)

In [10]:
# vectordb examples coming soon :)